In [ ]:
# from nba_api.stats.static import teams, players
# from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation
import pandas as pd
import numpy as np
import json
import difflib
import time
import requests

In [6]:
# Retry Wrapper 
def retry(func, retries=3):
    def retry_wrapper(*args, **kwargs):
        attempts = 0
        while attempts < retries:
            try:
                return func(*args, **kwargs)
            except requests.exceptions.RequestException as e:
                print(e)
                time.sleep(30)
                attempts += 1

    return retry_wrapper

In [13]:
# Get All Nba Team
from nba_api.stats.static import teams
nba_teams = teams.get_teams()
for i in nba_teams:
    if i['abbreviation'] == 'GSW':
        team_example = i
        break
team_example


{'id': 1610612744,
 'full_name': 'Golden State Warriors',
 'abbreviation': 'GSW',
 'nickname': 'Warriors',
 'city': 'Golden State',
 'state': 'California',
 'year_founded': 1946}

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder

gamefinder = leaguegamefinder.LeagueGameFinder(
    team_id_nullable=team_example['id'],
    # season_nullable = "2016-17"
    )

gamefinder
games = gamefinder.get_data_frames()[0]
# count Season
games.groupby(
    games.SEASON_ID.str[-4:]
    )[['GAME_ID']].count().loc['2024':]

# games.head()
# games['SEASON_ID'].unique() # ['42016', '22016', '12016']

In [65]:
# Query Example
# By Year
games.groupby(games.SEASON_ID.str[-4:])[['GAME_ID']].count().loc['2015':]
games_2024 = games[games.SEASON_ID.str[-4:] == '2024']

# By Team matchup
war_games_2024 = games_2024[games_2024.MATCHUP.str.contains('HOU')]

# Find the lastest game
# last_game = games_2024.sort_values('GAME_DATE').iloc[-4::]
last_game = games_2024.sort_values('GAME_DATE').iloc[-1]
last_game

SEASON_ID                            22024
TEAM_ID                         1610612744
TEAM_ABBREVIATION                      GSW
TEAM_NAME            Golden State Warriors
GAME_ID                         0022401163
GAME_DATE                       2025-04-09
MATCHUP                        GSW vs. SAS
WL                                       L
MIN                                    239
PTS                                    111
FGM                                     36
FGA                                     86
FG_PCT                               0.419
FG3M                                    16
FG3A                                  48.0
FG3_PCT                              0.333
FTM                                     23
FTA                                     28
FT_PCT                               0.821
OREB                                  16.0
DREB                                  29.0
REB                                   45.0
AST                                     29
STL        

In [78]:
from nba_api.stats.endpoints import playbyplayv2

pbp = playbyplayv2.PlayByPlayV2(last_game['GAME_ID'])
test = pbp.get_data_frames()[0]
with open('testoutput.txt','w') as f1:
    f1.write(test.iloc[-3::].to_markdown(
        index=False, 
        tablefmt="grid",
        stralign="center"
    ))
